In [1]:
from Selector import *
import sys
import time
from datetime import datetime
from view.HTML import show
import json
from indicators.MA import *
from indicators.Gap import *
# from indicators.Trend import *
from strategy.sequence.MAStrategy import *
from strategy.sequence.RStrategy import *
from simulation.Simulator import *
sys.path.append("../data")
import StockDao as sd
from view.ViewDataFactory import *
from strategy.sequence.StrategyS1 import *
from simulation.Simulator import *

In [2]:
def  up(list,i):
    k = list[i]
    pk = list[i-1]
    if (k.low - pk.high)/pk.high > 0.01 and k.high == k.close and k.high > k.low:
        return True
    return False
            

In [3]:
def wave(list,i):
    """
    """
    if len(list) < 40:
        return False
    current = i - 15
    lk = list[i]
    flag = False
    bk = None
    while current <= i:
        k = list[current]
        if not k.isUp():
            current = current + 1
            continue
        k5 = list[current-5]
        k10 = list[current-10]
        k15 = list[current-15]
        r5 = (k.close - k5.close)/k5.close
        r10 = (k.close - k10.close)/k10.close
        r15 = (k.close - k15.close)/k15.close
        if r5 < -0.35:
            return False
        if r10 < -0.35:
            return False
        if r15 < -0.35:
            return False
        if lk.low > k5.high and r5 > 0.4:
            flag = True
            bk = k5
        if lk.low > k10.high and r10 > 0.4:
            flag = True
            bk = k10
        if lk.low > k15.high and r15 > 0.4:
            flag = True
            bk = k15
        if (k.close - lk.close)/lk.close > 0.4:
            return False
        current = current + 1
        
    #如果当前跌得太多
    if flag and (k.close - bk.close)/bk.close < 0.2:
            return False
    
    return flag

In [4]:
def down(list,i):
    k = list[-1]
    if not k.isDown():
        return False
    j = i-1
    h = i - 4
    while j>=h:
        kc = list[j]
        if kc.isDown() or kc.open < k.close:
            return False
        j = j - 1
    return True
        

In [ ]:
def decide(list,i,trend,ma10,ma20,ma60,gap):
        """
        策略
        :param list: 测试数据
        :param trader: 交易者
        :param i: 当前在list中的位置
        :return:
        """
        k = list[i]
        pre = list[i-1]
        if gap.support(k.date,k.close,pre.minOC(),20) and k.isDown():
            return True,1
#         if up(list,i):
#             return True,2
#         if wave(list,i):
#             return True,3
        if down(list,i):
            return True,4
        return False,-1

In [ ]:
def firgueF(self,code):
    """显示股票"""
    #获取数据
    begin = "20160105"
    end = None
#     begin = "20150105"
#     end = '20160722'
    listK = sd.get_k_data(code,begin ,end)
    data,volume = ViewDataFactory.getCandlestick(listK)
    madic = ViewDataFactory.getMA(listK,(20,60))
    trendLine,trendMergeLine,trendBig = ViewDataFactory.getTrendLine(listK)
    
    #获取开始显示的时间
    mDate = 0
#     if len(data)>0:
#         if len(listK)/2>abs(-250):
#             mDate = data[-250][0]
#         else:
#             mDate = data[len(listK)/2][0]
    if len(data)>100:
        mDate = data[-100][0]
    else:
        mDate = data[len(data)][0]
            
    #显示
    params = {}
    params["code"] = code
    params["condition"] = self.condition.get(code,-100)
    params["mDate"] = mDate
    params["data"] = json.dumps(data)
    params["volume"] = json.dumps(volume)
    params["line20"] = json.dumps(madic[20].line)
    params["line60"] = json.dumps(madic[60].line)
    params["trendLine"] = json.dumps(trendLine)
    params["trendMergeLine"] = json.dumps(trendMergeLine)
    params["trendBig"] = json.dumps(trendBig)
    clear_output()
    show("Selector.html",params)
   


In [ ]:
class KB(Selector):
    """红绿红"""
    condition = {}
    def analyze(self,list):
        """分析股票"""
#         if len(list) < 50:
#             return False
        i = 0
        flag = False
        trend = Trend()
        ma10 = MA(10)
        ma20 = MA(20)
        ma60 = MA(60)
        gap = Gap(0.05)
        while i < len(list):
            k = list[i]
#             trend.add(k)
#             ma10.add(k.date,k.close)
#             ma20.add(k.date,k.close)
#             ma60.add(k.date,k.close)
            gap.fresh(k)
            i += 1
        flag,condition = decide(list,len(list)-1,trend,ma10,ma20,ma60,gap)
        if flag == True:
            self.condition[list[0].code] = condition
        return flag
    
    def firgue(self,code):
        """显示股票"""
        firgueF(self,code)


In [ ]:
kb = KB(True,'20180101')



selecting......


In [ ]:
for code in kb.codes:
    print(code.split('.')[0])